In [3]:
import shutil
import sys
sys.dont_write_bytecode = True
sys.path.append('/usr/lib/freecad-daily/lib/')
sys.path.append('/usr/lib/freecad/lib/')

import math
import FreeCAD
from FreeCAD import Base
import Part
import os
from collections import Counter
import collections
from collections import OrderedDict

import Mesh
import Draft
import MeshPart
import numpy as np
import heapq

import breeder_blanket_model_maker
import pkg_resources


In [19]:
def add_cooling_pipes_to_div(div_to_cool,channel_poloidal_height,channel_radial_height,plate_poloidal_height,plasma):

    middle, top_bottom = chop_top_and_bottom_from_cooling_plate(plate=div_to_cool,
                                                                channel_poloidal_height=channel_poloidal_height,
                                                                plate_poloidal_height=plate_poloidal_height)

   
    largest_face, largest_face_id = find_largest_face(div_to_cool)

    second_largest_face, second_largest_face_id = find_largest_face(div_to_cool,2)

    back_face = find_envelope_back_face(div_to_cool,plasma)
    back_face_id = find_envelope_back_face_id(div_to_cool,plasma)

    #print('face ids ',largest_face_id,second_largest_face_id,back_face_id)

    #faces_not_in_first_wall = [div_to_cool.Faces[largest_face_id],div_to_cool.Faces[second_largest_face_id],div_to_cool.Faces[back_face_id]]
    faces_not_in_first_wall = [largest_face,back_face,second_largest_face]
    #print(faces_not_in_first_wall)
    
    #print('poloidal_cooling_plate_mm',poloidal_cooling_plate_mm)

    step_list=[0]
    for c in range(34):#must be and even number to make sure pipe has both front back walls
        if c % 2 == 0:
            step_list.append(step_list[c]+10+c)
            #print(step_list[c]+10+(c*1.5))
        else:
            step_list.append(step_list[c]+channel_radial_height)
    #print('step_list',step_list)
    #print('faces_not_in_first_wall',faces_not_in_first_wall)

    list_of_cooling_pipes=[]
    list_of_structure=[]

    try:
        for counter, step in enumerate(step_list[1:]):
            #print('adding cooling channel ',counter)

            #cooling_solid = div_to_cool.makeThickness(faces_not_in_first_wall, -step, 0, True)
            cooling_solid = div_to_cool.makeThickness(faces_not_in_first_wall, -step, 0, True)
            #print(counter,cooling_solid.Volume)


            if counter==0:

                #save_list_of_solids_to_stp(cooling_solid, 'cooling_solid' + str(counter))
                cooling_solid = cooling_solid.cut(top_bottom)
                list_of_structure.append(cooling_solid)
            else:
                cut_solid= cooling_solid.cut(previous_solid)

                #save_list_of_solids_to_stp(cut_solid, 'cooling_solid' + str(counter))
                #all_solids = cooling_solid.fuse(previous_solid)

                if counter % 2 == 0:
                    cut_solid = cut_solid.cut(top_bottom)
                    list_of_structure.append(cut_solid)
                else:
                    cut_solid=cut_solid.cut(top_bottom)
                    list_of_cooling_pipes.append(cut_solid)


                print('cooling channel formation cut_solid.Volume',cut_solid.Volume)
                # if cut_solid.Volume < 3000000:
                #     print('cut volume is small so this is the last cooling channel')
                #     break

            previous_solid=cooling_solid

        list_of_structure.append(middle.cut(cooling_solid))
        list_of_structure.append(top_bottom)
        
    except:
        pass
        print('error the end of the cooling channels has been reached')


    
    return list_of_cooling_pipes,list_of_structure

In [30]:
def chop_top_and_bottom_from_cooling_plate(plate, channel_poloidal_height,plate_poloidal_height):

    #print('plate',plate)

    largest_face, largest_face_id=find_largest_face(plate)
    print(largest_face, largest_face_id)

    thickness_of_top_bottom_layers = (plate_poloidal_height-channel_poloidal_height)/2.0
    print('thickness_of_top_bottom_layers',thickness_of_top_bottom_layers)
    
#    poly_face.scale(2.0, poly_face.CenterOfMass)

    new_face1 = plate.Faces[largest_face_id].extrude(largest_face.normalAt(0, 0) * thickness_of_top_bottom_layers)

    new_face2 = plate.Faces[largest_face_id].extrude(largest_face.normalAt(0, 0) * -thickness_of_top_bottom_layers)

    top = new_face1.fuse(new_face2)
    print('top vol',top.Volume)
    
    top.exportStep('/home/jshim/detailed_HCLL/step/top.step')    

    new_face1 = plate.Faces[largest_face_id].extrude(largest_face.normalAt(0, 0) * -(channel_poloidal_height +thickness_of_top_bottom_layers ))

    new_face2 = plate.Faces[largest_face_id].extrude(largest_face.normalAt(0, 0) * (channel_poloidal_height +thickness_of_top_bottom_layers ))

    middle = new_face1.fuse(new_face2)
    print('mid vol',middle.Volume)
    
    middle.exportStep('/home/jshim/detailed_HCLL/step/middle.step')

    pre_skinny_div_to_cool = plate.common(middle)
    print('pre_skinny_div_to_cool vol',pre_skinny_div_to_cool.Volume)
    
    skinny_div_to_cool = pre_skinny_div_to_cool.cut(top)
    print('skinny_div_to_cool vol',skinny_div_to_cool.Volume)

    top_bottom = plate.cut(skinny_div_to_cool)

    return skinny_div_to_cool, top_bottom

In [17]:
def find_largest_face(solid_or_list_of_faces,n=1):
    if type(solid_or_list_of_faces) == list:
        #print('a list')
        list_of_faces = solid_or_list_of_faces
    else:
        #print('not a list')
        list_of_faces = solid_or_list_of_faces.Faces

    face_sizes=[]
    face_ids=[]
    for face_id, face in enumerate(list_of_faces):
        face_sizes.append(face.Area)
        face_ids.append(face_id)

    #print('face_sizes',face_sizes)
    largest_size = heapq.nlargest(n, face_sizes)[-1]

    index_to_return = face_sizes.index(largest_size)


    return list_of_faces[index_to_return] , face_ids[index_to_return]

In [4]:
file='/home/jshim/detailed_HCLL/step/slice_cooling_plate_material_mod13.step'
multipart_step = Part.read(file)

In [6]:
multipart_step.Volume

5963280.3720196085

In [31]:
middle, top_bottom =chop_top_and_bottom_from_cooling_plate(multipart_step,3.0,5.0)

(<Face object at 0x1f97040>, 7)
('thickness_of_top_bottom_layers', 1.0)
('top vol', 2109332.0452914066)
('mid vol', 8437328.18116572)
('pre_skinny_div_to_cool vol', 0.0)
('skinny_div_to_cool vol', 0.0)


In [14]:
middle.Volume

0.0

In [15]:
top_bottom.Volume

5963280.3720196085